<h2>1. Import Libraries</h2>

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
import time

<h2>2. SCRAPPING hotels infos and comments</h2>

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome("/home/med/TPS WEB scraping /chromedriver",options=chrome_options)

hotels= pd.read_csv('hotels.csv')
links=hotels['link']
all_data=[]
k=0
for url in links[99:110]:
    k=k+1
    driver.get(url)
    driver.get_screenshot_as_file("screenshot.png")

    #===================== stockage global des donnees==========================

    data= {}

    try:
        time.sleep(2)
        hotel_name = driver.find_element(By.ID,"HEADING")
        hotel_name =hotel_name.text
        time.sleep(3)
        data['hotel_name']=hotel_name
        print("========== hotel name success===============")
    except:
        data['hotel_name']=''

    #ranking
    try:
        XPath= "//*[@id='component_3']/div/div/div[1]/div[2]/a"
        ratee = driver.find_element(by=By.XPATH,value=XPath)
        ranking = ratee.find_element(by=By.XPATH,value='./span')
        text = ranking.get_attribute("class")
        data['rank']=text
        print("========== rank success===============")
    except:
        data['rank']



    try:
        XPath ="//*[@id='component_3']/div/div/div[1]/div[2]/a/span[2]"
        nb_review = driver.find_element(by=By.XPATH,value=XPath)
        nb_review = nb_review.text
        data['nb_reviews']=nb_review
        print("========== nb reviews success===============")
    except:
        data['nb_reviews']=''
    time.sleep(3)

    #number of hotels
    try:
   
        XPath="//*[@id='component_3']/div/div/div[1]/div[2]/div/a/span"
        nb_hotels = driver.find_element(by=By.XPATH, value=XPath)
        nb_hotels=nb_hotels.text
        data['hotel_number']=nb_hotels
        print("==========nb hotels success===============")
    except:
        data['hotel_number']=''

    time.sleep(3)
    #localisation
    try:
        XPath = "//*[@id='component_3']/div/div/div[2]/div/div[2]/div/div[1]/div/span[2]/span"
        localisation = driver.find_element(by=By.XPATH,value=XPath)
        localisation = localisation.text
        data['localisation']= localisation
        print("========== localisation success===============")
    except:
        data['localisation']=''

    time.sleep(3)
    #hotel_link
    try:
        XPath = "//*[@id='component_3']/div/div/div[1]/div[3]/div/a"
        link =driver.find_element(By.XPATH, value=XPath)
        hotel_link= link.get_property('href')
        data['hotel_link']=hotel_link
        print("========== hotel link success===============")
    except:
        data['hotel_link']=''

    time.sleep(3)
    #phone
    try:
        XPath="//*[@id='component_3']/div/div/div[2]/div/div[2]/div/div[2]/div/a/span[2]"
        #XPath= "//div[contains(@class,'eeVey S4 H3 f u LGJIs')]"
        phone = driver.find_element(By.XPATH, value=XPath)
        phone = phone.text
        data['phone']=phone
        print("========== hotel phone success===============")
    except:
        data['phone']=''
        

    #==================Property amenities==========================
    time.sleep(3)
    
    #amenities
    try:
        XPath = "//*[@id='ABOUT_TAB']/div[2]/div[2]/div[1]/div[2]"
        amenities = driver.find_element(By.XPATH, value=XPath)
        amen = amenities.text
        amenities= amen.split('\n')
        data['amenities']=amenities
    except:
        data['amenities']=''
    print("========== amenities success===============")
    time.sleep(3)
    #==================room features=========================

    try:
        XPath = "//*[@id='ABOUT_TAB']/div[2]/div[2]/div[1]/div[5]"
        feature = driver.find_element(By.XPATH, value=XPath)
        room = feature.text
        features= room.split('\n')
        data['room_features']=features
    except:
        data['room_features']=''
    print("========== room features success===============")

    time.sleep(3)



    #==================room types=========================

    try:
        XPath = "//*[@id='ABOUT_TAB']/div[2]/div[2]/div[1]/div[8]"
        types= driver.find_element(By.XPATH, value=XPath)
        rtype = types.text
        rtype= rtype.split('\n')

        data['room_type']=rtype
    except:
        data['room_type']=''


    all_comments=[]
    #========================== commments part ====================
    #=========================== scrap comments=================================
    stop=False

    number_of_comments =0
    while stop==False:
      time.sleep(5)
      
      XPath= "//div[contains(@class,'YibKl MC R2 Gi z Z BB pBbQr')]"
      comment_title = driver.find_elements(by=By.XPATH, value=XPath)
      

      n=len(comment_title)
      number_of_comments += n
      print("number of comment: ",number_of_comments)
      for i in range(n):
        whole_comment ={}
        print(f"comment number {i}...")
        a=comment_title[i]
        #hotel name
        whole_comment['hotel_name']= hotel_name
      #user_comment
        try:
            user_comment = a.find_element(by=By.XPATH,value="./div[1]/div/div[2]/span") 
            user_comment=user_comment.text

            whole_comment['user_name']=user_comment
        except:
            pass


        try:
            d= a.find_elements(by=By.XPATH,value="./div[1]/div/div[3]/span")
            li=[] 
            for i in range(len(d)):
                try:
                    li.append(d[i].text)
                    whole_comment['info_contr']=li
                except:
                    pass
        except:
            pass
        

        #comment rank
        
        try:
            XPath="//*[@id='component_16']/div/div[3]/div[3]/div[2]/div[1]/div/span"
            com_rate = driver.find_element(by=By.XPATH,value=XPath)
            com_rank = com_rate.find_element(by=By.XPATH,value='./span')
            text = com_rank.get_attribute("class")
            whole_comment['comment_rank']=text
            print("==========comment  rank success===============")
        except:
            pass

        try:
           
            XPath ="//*[@id='component_16']/div/div[3]/div[4]/div[2]/div[2]/div/span"
            com_rate = driver.find_element(by=By.XPATH,value=XPath)
            com_rank = com_rate.find_element(by=By.XPATH,value='./span')
            text = com_rank.get_attribute("class")
            whole_comment['comment_rank']=text
            print("==========comment  rank success===============")
        except:
            pass
        
        try:
            XPath ="//*[@id='component_16']/div/div[3]/div[5]/div[3]/div[1]/div/span"
            
            com_rate = driver.find_element(by=By.XPATH,value=XPath)
            com_rank = com_rate.find_element(by=By.XPATH,value='./span')
            text = com_rank.get_attribute("class")
            whole_comment['comment_rank']=text
            print("==========comment  rank success===============")
        except:
            pass

        #comment title
        
        try:
            f= a.find_elements(by=By.XPATH,value="./div[2]/div[2]/a/span/span")
            coment_title=f[0].text
            whole_comment['comment_title']=coment_title
        except:
            pass

        
        #comment paragraphe
        try:
            comment= a.find_elements(by=By.XPATH,value="./div[2]/div[3]/div[1]/div[1]/q/span")
            comment=comment[0].text
            whole_comment['comment_text']=comment
        except:
            pass

        all_comments.append(whole_comment)



      if number_of_comments==300:
        break

      time.sleep(3)
      try:
          
          XPath= "//a[contains(@class,'ui_button nav next primary')]"

          driver.find_element(by=By.XPATH, value=XPath).click()


      except:
        stop=True

    data.update({'all_comments':all_comments})
    all_data.append(data)
    print(f"=========================== hotel number {k} is done ======================")

/tmp/ipykernel_5014/2253858798.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/home/med/TPS WEB scraping /chromedriver",options=chrome_options)


========== hotel name success===============
========== rank success===============
========== nb reviews success===============
==========nb hotels success===============
========== localisation success===============
========== hotel link success===============
========== hotel phone success===============
========== amenities success===============
========== room features success===============
number of comment:  10
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  20
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
=========================== hotel number 1 is done ======================
========== hotel name success===============
========== rank success==========

========== nb reviews success===============
==========nb hotels success===============
========== localisation success===============
========== amenities success===============
========== room features success===============
number of comment:  10
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  20
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  30
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  40
comment number 0...
comment number 1...
comment number 2...
comment number 3...
c

comment number 9...
number of comment:  50
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  60
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  70
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  80
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  90
comment number 0...
comment number 1...
comment number 2...
comme

comment number 9...
number of comment:  100
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  110
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  120
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  130
comment number 0...
comment number 1...
comment number 2...
comment number 3...
comment number 4...
comment number 5...
comment number 6...
comment number 7...
comment number 8...
comment number 9...
number of comment:  140
comment number 0...
comment number 1...
comment number 2...


KeyboardInterrupt: 

In [17]:

all_data2.to_csv('data.csv')